In [1]:
import pandas as pd
import numpy as np
import os
import utils

features_1 = [
    'sx_x',
    'sx_y',
    'sx_z',
    'sx_roll',
    'sx_pitch',
    'sx_yaw',
    'sx_thumb',
    'sx_forefinger',
    'sx_middle_finger',
    'sx_ring_finger',
    'sx_little_finger',
    'dx_x',
    'dx_y',
    'dx_z',
    'dx_roll',
    'dx_pitch',
    'dx_yaw',
    'dx_thumb',
    'dx_forefinger',
    'dx_middle_finger',
    'dx_ring_finger',
    'dx_little_finger'
    ]



In [2]:
path = "./tctodd/"
#path = "../../Desktop/MML Project/tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

In [3]:
df = pd.DataFrame(data, columns=['label', 'time_series'])
seed = 0

In [4]:
X = df['time_series']
y = df['label']

n_coords = 60
X_interp = utils.interpolate_data(X, n_coords)
X_interp_concatenated = utils.concatenate_examples(X_interp)

print(f"Shape of new interpolated and stacked X is {X_interp.shape}")

Shape of new interpolated and stacked X is (2565,)


In [5]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_interp_concatenated, y,
                                                    stratify=y, 
                                                    test_size=0.1, random_state=seed)

ts_forest = TimeSeriesForestClassifier(random_state=seed)

ts_forest.fit(X_train, y_train)
preds = ts_forest.predict(X_test)
score = accuracy_score(y_test, preds)
print(f"Score of the plain TimeSeriesForestClassifier without cross-validation is {score}")

Score of the plain TimeSeriesForestClassifier without cross-validation is 0.6264591439688716
